# Library

In [1]:
import cv2 as cv
import numpy as np
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.FaceMeshModule import FaceMeshDetector

-------------

# Eye Landmark

In [2]:
LEFT_EYE =[362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]

# Facedetector object

In [3]:
detector= FaceDetector()
meshdetector= FaceMeshDetector(maxFaces=1)

I0000 00:00:1739449552.731203   38669 gl_context.cc:369] GL version: 2.1 (2.1 INTEL-10.4.14), renderer: Intel HD Graphics 3000 OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1739449552.741773   38737 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1739449552.765992   38669 gl_context.cc:369] GL version: 2.1 (2.1 INTEL-10.4.14), renderer: Intel HD Graphics 3000 OpenGL Engine
W0000 00:00:1739449552.775636   38745 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


-------------------

# Select video file or webcam

In [4]:
webcam= True
video_path= 'video.mp4'

W0000 00:00:1739449552.800877   38745 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# Run program

In [5]:
if webcam:
    cap= cv.VideoCapture(0)
else:
    cap= cv.VideoCapture(video_path)

if (cap.isOpened()== False):
    print("Error opening video stream or file")

In [ ]:
while(cap.isOpened()):
    ret, frame= cap.read()
    ret, frame2= cap.read()
    if ret == True:
        face_img, bbox= detector.findFaces(frame)
        face_img, faces= meshdetector.findFaceMesh(frame)

        if bbox:
            center= bbox[0]["center"]   
            if faces:
                left_eye_points= np.array([[faces[0][p][0],faces[0][p][1]] for p in LEFT_EYE])
                (ex,ey,ew,eh)= cv.boundingRect(left_eye_points)
                eye_roi= frame2[ey:ey+eh, ex:ex+ew]
                eye_roi_gr= cv.cvtColor(eye_roi, cv.COLOR_BGR2GRAY)
                _, iris= cv.threshold(eye_roi_gr, 40, 255, cv.THRESH_BINARY_INV)
                contours, _= cv.findContours(iris, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
                contours= sorted(contours, key= lambda x: cv.contourArea(x), reverse= True)

                if contours:
                    (ix,iy,iw,ih)= cv.boundingRect(contours[0])
                    ix_cntr, iy_centr= ix+int(iw/2) + ex, iy+int(ih/2)+ey
                    cv.circle(frame2, (ix_cntr, iy_centr), 5, (0,255,0), -1)

                    ix_cntr_e, iy_centr_e= ix+int(iw/2), iy+int(ih/2)

                    if ix_cntr_e > int(2*ew/3):
                        text= "right"
                    elif ix_cntr_e < int(ew/3):
                        text= "left"
                    else:
                        text= "center"

                    (w, h), _= cv.getTextSize(text, cv.FONT_HERSHEY_PLAIN, 3, 2)
                    cv.rectangle(frame2, (100, 100 - h), (100 + w, 100 + h), (0, 60, 0), -1)
                    cv.putText(frame2, text, (100, 100), cv.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 2)

        cv.imshow('frame2', frame2)
        if cv.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv.destroyAllWindows()

W0000 00:00:1739449555.227146   38747 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
